In [1]:
# Importación de librerías necesarias

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from neuralprophet import NeuralProphet, set_log_level
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import logging
import warnings
import sys
#
warnings.filterwarnings("ignore")
# Configurar logging y warnings
logging.basicConfig(level=logging.WARNING, format='%(asctime)s - %(levelname)s - %(message)s')
warnings.filterwarnings('ignore')
set_log_level("ERROR")

In [2]:
# 1. Cargar los datos
print("Cargando datos...")
df = pd.read_csv('data/df_final_para_neuralprophet.csv')

# 2. Preparación de los datos
print("Preparando datos...")
df['ds'] = pd.to_datetime(df['ds'])
df.sort_values(['product_id', 'customer_id', 'ds'], inplace=True)

# 3. Agregación de datos
print("Agregando datos...")
df_agg = df.groupby(['product_id', 'ds']).agg({
    'y': 'sum',
    'customer_id': 'nunique',
    'año': 'first',
    'mes': 'first',
    'trimestre': 'first',
    'cat1_encoded': 'first',
    'cat2_encoded': 'first',
    'cat3_encoded': 'first',
    'brand_encoded': 'first',
    'avg_tn_por_producto': 'mean',
    'avg_qty_por_cliente': 'mean',
    'ratio_tn_qty': 'mean',
    'precios_cuidados': 'mean',
    'sku_size': 'mean',
    'avg_size_por_brand': 'mean',
    'max_size_por_brand': 'mean',
    'min_size_por_brand': 'mean',
    'size_range_por_brand': 'mean'
}).reset_index()

df_agg

Cargando datos...
Preparando datos...
Agregando datos...


,product_id,ds,y,customer_id,año,mes,trimestre,cat1_encoded,cat2_encoded,cat3_encoded,brand_encoded,avg_tn_por_producto,avg_qty_por_cliente,ratio_tn_qty,precios_cuidados,sku_size,avg_size_por_brand,max_size_por_brand,min_size_por_brand,size_range_por_brand
0,20001,2017-01-01,934.77222,186,2017,1,1,1,10,47,0,8.156253,1.910679,1.018760,0.0,3000.0,1834.663381,3000.0,400.0,2600.0
1,20001,2017-02-01,798.01620,185,2017,2,1,1,10,47,0,8.156253,1.947834,1.053940,0.0,3000.0,1834.663381,3000.0,400.0,2600.0
2,20001,2017-03-01,1303.35771,188,2017,3,1,1,10,47,0,8.156253,1.919275,1.680207,0.0,3000.0,1834.663381,3000.0,400.0,2600.0
3,20001,2017-04-01,1069.96130,104,2017,4,2,1,10,47,0,8.156253,2.472872,2.581326,0.0,3000.0,1834.663381,3000.0,400.0,2600.0
4,20001,2017-05-01,1502.20132,238,2017,5,2,1,10,47,0,8.156253,1.758276,1.403028,0.0,3000.0,1834.663381,3000.0,400.0,2600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22344,21276,2019-08-01,0.01265,4,2019,8,3,2,6,18,23,0.007101,4.372053,0.001860,0.0,140.0,201.614133,750.0,10.0,740.0
22345,21276,2019-09-01,0.01856,7,2019,9,3,2,6,18,23,0.007101,3.528196,0.001855,0.0,140.0,201.614133,750.0,10.0,740.0
22346,21276,2019-10-01,0.02079,8,2019,10,4,2,6,18,23,0.007101,3.438649,0.001593,0.0,140.0,201.614133,750.0,10.0,740.0
22347,21276,2019-11-01,0.03341,8,2019,11,4,2,6,18,23,0.007101,3.120522,0.002506,0.0,140.0,201.614133,750.0,10.0,740.0


In [3]:
# 4. Función para filtrar datos por rango de product_id
def filtrar_por_producto(df, producto_inicio, producto_fin):
    return df[(df['product_id'] >= producto_inicio) & (df['product_id'] <= producto_fin)]

# 5. Filtrado de datos para el rango de productos deseado
producto_inicio = 20001
producto_fin = 20012  # Puedes ajustar esto según tus necesidades. Ultimo = 21276
datos_filtrados = filtrar_por_producto(df_agg, producto_inicio, producto_fin)

# Preparación de datos para NeuralProphet
print(datos_filtrados)
datos_filtrados.info()

     product_id         ds           y  customer_id   año  mes  trimestre  \
0         20001 2017-01-01   934.77222          186  2017    1          1   
1         20001 2017-02-01   798.01620          185  2017    2          1   
2         20001 2017-03-01  1303.35771          188  2017    3          1   
3         20001 2017-04-01  1069.96130          104  2017    4          2   
4         20001 2017-05-01  1502.20132          238  2017    5          2   
..          ...        ...         ...          ...   ...  ...        ...   
427       20012 2019-08-01   301.48373          122  2019    8          3   
428       20012 2019-09-01   330.62376          157  2019    9          3   
429       20012 2019-10-01   282.29947          111  2019   10          4   
430       20012 2019-11-01   357.11018          172  2019   11          4   
431       20012 2019-12-01   173.13004           75  2019   12          4   

     cat1_encoded  cat2_encoded  cat3_encoded  brand_encoded  \
0          

In [4]:
# 6. Agregar registros en cero
print("Agregando registros en cero...")
def agregar_registros_cero(df):
    df_list = []
    for producto, datos_producto in df.groupby('product_id'):
        fecha_inicio = datos_producto['ds'].min()
        fecha_fin = datos_producto['ds'].max()
        rango_fechas = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='MS')
        datos_completos = pd.DataFrame(rango_fechas, columns=['ds'])
        datos_completos['product_id'] = producto
        datos_completos = datos_completos.merge(datos_producto, on=['ds', 'product_id'], how='left')
        
        # Llenar los valores faltantes
        datos_completos['y'].fillna(0, inplace=True)
        for col in datos_completos.columns:
            if col not in ['ds', 'product_id', 'y']:
                datos_completos[col].fillna(datos_completos[col].mode().iloc[0], inplace=True)
        
        df_list.append(datos_completos)
    return pd.concat(df_list, ignore_index=True)

# Aplicar la función para agregar registros en cero
datos_completos_cero = agregar_registros_cero(datos_filtrados)

# Mostrar cantidad de registros
print(f"Cantidad de registros en datos_filtrados: {len(datos_filtrados)}")
print(f"Cantidad de registros en datos_completos_cero: {len(datos_completos_cero)}")

# Verificar que todas las columnas necesarias estén presentes
print("\nColumnas en datos_completos_cero:")
print(datos_completos_cero.columns)

# Verificar los tipos de datos
print("\nTipos de datos en datos_completos_cero:")
print(datos_completos_cero.dtypes)

# Verificar si hay valores nulos
print("\nValores nulos en datos_completos_cero:")
print(datos_completos_cero.isnull().sum())

# Mostrar las primeras filas del DataFrame final
print("\nPrimeras filas de datos_completos_cero:")
print(datos_completos_cero.head())

Agregando registros en cero...
Cantidad de registros en datos_filtrados: 432
Cantidad de registros en datos_completos_cero: 432

Columnas en datos_completos_cero:
Index(['ds', 'product_id', 'y', 'customer_id', 'año', 'mes', 'trimestre',
       'cat1_encoded', 'cat2_encoded', 'cat3_encoded', 'brand_encoded',
       'avg_tn_por_producto', 'avg_qty_por_cliente', 'ratio_tn_qty',
       'precios_cuidados', 'sku_size', 'avg_size_por_brand',
       'max_size_por_brand', 'min_size_por_brand', 'size_range_por_brand'],
      dtype='object')

Tipos de datos en datos_completos_cero:
ds                      datetime64[ns]
product_id                       int64
y                              float64
customer_id                      int64
año                              int64
mes                              int64
trimestre                        int64
cat1_encoded                     int64
cat2_encoded                     int64
cat3_encoded                     int64
brand_encoded                   

In [5]:
# Definir las operaciones de agregación para cada columna
operaciones = {
    'y': 'sum',
    'customer_id': 'nunique',
    'año': 'first',
    'mes': 'first',
    'trimestre': 'first',
    'cat1_encoded': 'first',
    'cat2_encoded': 'first',
    'cat3_encoded': 'first',
    'brand_encoded': 'first',
    'avg_tn_por_producto': 'mean',
    'avg_qty_por_cliente': 'mean',
    'ratio_tn_qty': 'mean',
    'precios_cuidados': 'max',
    'sku_size': 'first',
    'avg_size_por_brand': 'first',
    'max_size_por_brand': 'first',
    'min_size_por_brand': 'first',
    'size_range_por_brand': 'first'
}

# Realizar la sumarización
datos_sumarizados = datos_completos_cero.groupby(['ds', 'product_id']).agg(operaciones).reset_index()

# Renombrar la columna 'y' a 'total_TN' para mayor claridad
datos_sumarizados = datos_sumarizados.rename(columns={'y': 'total_TN'})

# Función para categorizar el tamaño del producto
def categorizar_tamaño(row):
    if row['sku_size'] <= row['min_size_por_brand']:
        return '1'
    elif row['sku_size'] >= row['max_size_por_brand']:
        return '3'
    else:
        return '2'

# Agregar la columna de categoría de tamaño
datos_sumarizados['tamaño_categoria'] = datos_sumarizados.apply(categorizar_tamaño, axis=1)
# Eliminar la columna customer_id
datos_sumarizados = datos_sumarizados.drop(columns=['customer_id'])


# Verificar el resultado
print("Forma del DataFrame sumarizado:", datos_sumarizados.shape)
print("\nColumnas del DataFrame sumarizado:")
print(datos_sumarizados.columns)
print("\nPrimeras filas del DataFrame sumarizado:")
print(datos_sumarizados.head())
print("\nEstadísticas descriptivas del DataFrame sumarizado:")
print(datos_sumarizados.describe())
print("\nValores nulos en el DataFrame sumarizado:")
print(datos_sumarizados.isnull().sum())

# Guardar el DataFrame sumarizado en un archivo CSV
datos_sumarizados.to_csv('datos_sumarizados_por_fecha_y_producto.csv', index=False)
print("\nDataFrame sumarizado guardado en 'datos_sumarizados_por_fecha_y_producto.csv'")

Forma del DataFrame sumarizado: (432, 20)

Columnas del DataFrame sumarizado:
Index(['ds', 'product_id', 'total_TN', 'año', 'mes', 'trimestre',
       'cat1_encoded', 'cat2_encoded', 'cat3_encoded', 'brand_encoded',
       'avg_tn_por_producto', 'avg_qty_por_cliente', 'ratio_tn_qty',
       'precios_cuidados', 'sku_size', 'avg_size_por_brand',
       'max_size_por_brand', 'min_size_por_brand', 'size_range_por_brand',
       'tamaño_categoria'],
      dtype='object')

Primeras filas del DataFrame sumarizado:
          ds  product_id    total_TN   año  mes  trimestre  cat1_encoded  \
0 2017-01-01       20001   934.77222  2017    1          1             1   
1 2017-01-01       20002   550.15707  2017    1          1             1   
2 2017-01-01       20003  1063.45835  2017    1          1             0   
3 2017-01-01       20004   555.91614  2017    1          1             0   
4 2017-01-01       20005   494.27011  2017    1          1             0   

   cat2_encoded  cat3_encoded 

In [6]:
# Cargar el archivo CSV
print("Cargando datos...")
df = pd.read_csv('datos_sumarizados_por_fecha_y_producto.csv')

# Convertir la columna 'ds' a datetime
df['ds'] = pd.to_datetime(df['ds'])

# Definir las operaciones de agregación para cada columna
operaciones = {
    'total_TN': 'sum',
    'año': 'first',
    'mes': 'first',
    'trimestre': 'first',
    'cat1_encoded': 'first',
    'cat2_encoded': 'first',
    'cat3_encoded': 'first',
    'brand_encoded': 'first',
    'avg_tn_por_producto': 'mean',
    'avg_qty_por_cliente': 'mean',
    'ratio_tn_qty': 'mean',
    'precios_cuidados': 'max',
    'sku_size': 'mean',
    'avg_size_por_brand': 'mean',
    'max_size_por_brand': 'max',
    'min_size_por_brand': 'min',
    'size_range_por_brand': 'mean',
    'tamaño_categoria': lambda x: x.mode().iloc[0] if not x.empty else np.nan
}

# Realizar la sumarización
print("Sumarizando datos...")
datos_sumarizados = df.groupby(['ds', 'product_id']).agg(operaciones).reset_index()

# Eliminar la columna customer_id si existe
if 'customer_id' in datos_sumarizados.columns:
    datos_sumarizados = datos_sumarizados.drop('customer_id', axis=1)

# Verificar el resultado
print("\nForma del DataFrame sumarizado:", datos_sumarizados.shape)
print("\nColumnas del DataFrame sumarizado:")
print(datos_sumarizados.columns)
print("\nPrimeras filas del DataFrame sumarizado:")
print(datos_sumarizados.head())
print("\nEstadísticas descriptivas del DataFrame sumarizado:")
print(datos_sumarizados.describe())
print("\nValores nulos en el DataFrame sumarizado:")
print(datos_sumarizados.isnull().sum())

# Guardar el DataFrame sumarizado en un nuevo archivo CSV
datos_sumarizados.to_csv('datos_sumarizados_final.csv', index=False)
print("\nDataFrame sumarizado guardado en 'datos_sumarizados_final.csv'")

# Definir los regresores
regresores = [
    'año', 'mes', 'trimestre', 'cat1_encoded', 'cat2_encoded', 'cat3_encoded',
    'brand_encoded', 'avg_tn_por_producto', 'avg_qty_por_cliente', 'ratio_tn_qty',
    'precios_cuidados', 'sku_size', 'avg_size_por_brand', 'max_size_por_brand',
    'min_size_por_brand', 'size_range_por_brand', 'tamaño_categoria'
]

print("\nRegresores disponibles:")
print(regresores)

# Verificar que todos los regresores estén presentes en el DataFrame
regresores_faltantes = [reg for reg in regresores if reg not in datos_sumarizados.columns]
if regresores_faltantes:
    print("\nAdvertencia: Los siguientes regresores no están presentes en el DataFrame:")
    print(regresores_faltantes)
else:
    print("\nTodos los regresores están presentes en el DataFrame.")

Cargando datos...
Sumarizando datos...

Forma del DataFrame sumarizado: (432, 20)

Columnas del DataFrame sumarizado:
Index(['ds', 'product_id', 'total_TN', 'año', 'mes', 'trimestre',
       'cat1_encoded', 'cat2_encoded', 'cat3_encoded', 'brand_encoded',
       'avg_tn_por_producto', 'avg_qty_por_cliente', 'ratio_tn_qty',
       'precios_cuidados', 'sku_size', 'avg_size_por_brand',
       'max_size_por_brand', 'min_size_por_brand', 'size_range_por_brand',
       'tamaño_categoria'],
      dtype='object')

Primeras filas del DataFrame sumarizado:
          ds  product_id    total_TN   año  mes  trimestre  cat1_encoded  \
0 2017-01-01       20001   934.77222  2017    1          1             1   
1 2017-01-01       20002   550.15707  2017    1          1             1   
2 2017-01-01       20003  1063.45835  2017    1          1             0   
3 2017-01-01       20004   555.91614  2017    1          1             0   
4 2017-01-01       20005   494.27011  2017    1          1         

In [7]:
print(datos_sumarizados.columns)
datos_sumarizados

Index(['ds', 'product_id', 'total_TN', 'año', 'mes', 'trimestre',
       'cat1_encoded', 'cat2_encoded', 'cat3_encoded', 'brand_encoded',
       'avg_tn_por_producto', 'avg_qty_por_cliente', 'ratio_tn_qty',
       'precios_cuidados', 'sku_size', 'avg_size_por_brand',
       'max_size_por_brand', 'min_size_por_brand', 'size_range_por_brand',
       'tamaño_categoria'],
      dtype='object')


,ds,product_id,total_TN,año,mes,trimestre,cat1_encoded,cat2_encoded,cat3_encoded,brand_encoded,avg_tn_por_producto,avg_qty_por_cliente,ratio_tn_qty,precios_cuidados,sku_size,avg_size_por_brand,max_size_por_brand,min_size_por_brand,size_range_por_brand,tamaño_categoria
0,2017-01-01,20001,934.77222,2017,1,1,1,10,47,0,8.156253,1.910679,1.018760,0.0,3000.0,1834.663381,3000.0,400.0,2600.0,3
1,2017-01-01,20002,550.15707,2017,1,1,1,10,47,15,6.056209,2.022002,0.960741,0.0,3000.0,1245.871918,10000.0,120.0,9880.0,2
2,2017-01-01,20003,1063.45835,2017,1,1,0,0,48,22,4.711343,1.744776,1.809341,0.0,475.0,365.348880,1000.0,60.0,940.0,2
3,2017-01-01,20004,555.91614,2017,1,1,0,0,48,22,3.386770,1.798512,1.337207,0.0,240.0,365.348880,1000.0,60.0,940.0,2
4,2017-01-01,20005,494.27011,2017,1,1,0,0,48,22,3.923400,1.862131,1.398542,0.0,120.0,365.348880,1000.0,60.0,940.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,2019-12-01,20008,195.36854,2019,12,4,1,14,53,15,3.091321,2.280424,1.194656,0.0,750.0,1245.871918,10000.0,120.0,9880.0,2
428,2019-12-01,20009,495.03574,2019,12,4,1,10,47,25,3.018410,2.215033,1.393536,0.0,3000.0,1659.303624,10000.0,400.0,9600.0,2
429,2019-12-01,20010,359.59998,2019,12,4,1,10,60,15,4.049247,2.081314,1.871189,0.0,400.0,1245.871918,10000.0,120.0,9880.0,2
430,2019-12-01,20011,392.38290,2019,12,4,1,10,60,15,3.448843,2.045489,0.989774,0.0,3000.0,1245.871918,10000.0,120.0,9880.0,2


In [9]:
# Suponemos que 'datos' ya está preparado y contiene las columnas ['ds', 'y', 'product_id']
datos['ds'] = pd.to_datetime(datos['ds'])
datos.sort_values(by=['product_id', 'ds'], inplace=True)

# Filtrar productos con más de 15 periodos de historia
productos_mas_historia = datos_completos_cero['product_id'].value_counts()[datos_completos_cero['product_id'].value_counts() > 15].index.tolist()
print(f"Número de productos con más de 15 periodos de historia: {len(productos_mas_historia)}")

# Dividir los datos en train, validation y test
train_df = datos_completos_cero[datos_completos_cero['ds'] < '2019-11-01']
val_df = datos_completos_cero[datos_completos_cero['ds'] == '2019-11-01']
test_df = datos_completos_cero[datos_completos_cero['ds'] == '2019-12-01']

def entrenar_y_predecir(train_df, val_df, test_df):
    modelos = {}
    predicciones = pd.DataFrame()
    for producto in tqdm(productos_mas_historia, desc="Procesando productos"):
        df_producto_train = train_df[train_df['product_id'] == producto][['ds', 'y']]
        df_producto_val = val_df[val_df['product_id'] == producto][['ds', 'y']]
        df_producto_test = test_df[test_df['product_id'] == producto][['ds', 'y']]
        
        m = NeuralProphet(
            n_forecasts=2,
            yearly_seasonality=True,
            batch_size=12,
            epochs=1000,
            learning_rate=0.025,
            loss_func='Huber',
            daily_seasonality=False,
            weekly_seasonality=False,
            seasonality_mode='multiplicative',
            # Agregado
            #lagged_reg_layers=[16, 8],
        )
        
        # Combinar train y validation para el entrenamiento final
        df_producto_train_val = pd.concat([df_producto_train, df_producto_val])
        
        m.fit(df_producto_train, validation_df=df_producto_val, freq='M')
        
        # Crear future dataframe manualmente
        last_date = df_producto_test['ds'].max()
        future_dates = pd.date_range(start=last_date, periods=3, freq='MS')[1:]  # Dos meses adelante
        future = pd.DataFrame({'ds': future_dates, 'y': [None]*len(future_dates)})
        
        forecast = m.predict(future)
        
        forecast.loc[forecast['yhat1'] < 0, 'yhat1'] = 0  # Convertir valores negativos a cero
        pred_febrero_2020 = forecast[forecast['ds'] == pd.Timestamp('2020-02-01')][['ds', 'yhat1']]
        pred_febrero_2020['product_id'] = producto
        predicciones = pd.concat([predicciones, pred_febrero_2020[['ds', 'yhat1', 'product_id']]])
        modelos[producto] = m
    return predicciones, modelos

# Entrenar el modelo y obtener predicciones
predicciones_febrero_2020, modelos = entrenar_y_predecir(train_df, val_df, test_df)

# Renombrar las columnas para el archivo de salida
predicciones_febrero_2020.rename(columns={'yhat1': 'tn'}, inplace=True)

# Convertir la columna 'ds' al formato YYYYMM
predicciones_febrero_2020['ds'] = predicciones_febrero_2020['ds'].dt.strftime('%Y%m')

# Guardar el resultado en un archivo .csv solo con la predicción para febrero 2020
predicciones_febrero_2020.to_csv('NeuralProphet_mas_20_periodos.v7.1.1.csv', index=False)

# Mostrar un resumen del resultado
print(f"Número total de predicciones: {len(predicciones_febrero_2020)}")
print(predicciones_febrero_2020.head())
#########################
#########################
#########################

from neuralprophet import NeuralProphet
import pandas as pd

# Renombrar la columna 'total_TN' a 'y'
datos_sumarizados = datos_sumarizados.rename(columns={'total_TN': 'y'})

# Definir las columnas necesarias para NeuralProphet
expected_columns = ['ds', 'y']

# Filtrar datos por product_id
product_ids = datos_sumarizados['product_id'].unique()

# Preparar un DataFrame para almacenar los resultados
resultados = pd.DataFrame()

# Entrenar y predecir para cada product_id
for pid in product_ids:
    # Crear una nueva instancia del modelo para cada product_id
    model = NeuralProphet(
        n_lags=3,  # Número de lags para los regresores
        n_forecasts=2,  # Número de períodos a predecir
    )
    
    datos_producto = datos_sumarizados[datos_sumarizados['product_id'] == pid].copy()
    
    # Filtrar las columnas necesarias
    datos_producto = datos_producto[expected_columns]
    
    # Asegurarse de que 'ds' sea de tipo datetime
    datos_producto['ds'] = pd.to_datetime(datos_producto['ds'])
    
    # Ordenar los datos por fecha
    datos_producto = datos_producto.sort_values('ds')
    
    # Verificar si hay suficientes datos para entrenar el modelo
    if len(datos_producto) <= 3:
        print(f"Not enough data for product_id {pid}")
        continue
    
    # Entrenar el modelo
    model.fit(datos_producto, freq='M')
    
    # Crear DataFrame futuro
    future = model.make_future_dataframe(df=datos_producto, periods=2)
    
    # Predecir
    forecast = model.predict(future)
    
    # Verificar las predicciones
    if forecast.empty:
        print(f"No predictions for product_id {pid}")
        continue
    
    # Filtrar predicciones para febrero de 2020
    predicciones_febrero = forecast[forecast['ds'].dt.strftime('%Y-%m') == '2020-02'][['ds', 'yhat1']]
    
    # Verificar si hay predicciones para febrero de 2020
    if predicciones_febrero.empty:
        print(f"No predictions for February 2020 for product_id {pid}")
        continue
    
    # Agregar product_id a las predicciones
    predicciones_febrero['product_id'] = pid
    
    # Renombrar la columna de predicción
    predicciones_febrero.rename(columns={'yhat1': 'tn'}, inplace=True)
    
    # Añadir las predicciones al DataFrame de resultados
    resultados = pd.concat([resultados, predicciones_febrero])

# Guardar los resultados en un archivo CSV
resultados[['product_id', 'tn']].to_csv('predicciones_febrero_2020.v9.0.0.csv', index=False)


Finding best initial lr:   0%|          | 0/203 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

Predicting: 4it [00:00, ?it/s]

Finding best initial lr:   0%|          | 0/203 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

...

Training: 0it [00:00, ?it/s]

Predicting: 4it [00:00, ?it/s]

Finding best initial lr:   0%|          | 0/203 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

Predicting: 4it [00:00, ?it/s]

In [8]:
from neuralprophet import NeuralProphet
import pandas as pd

# Renombrar la columna 'total_TN' a 'y'
datos_sumarizados = datos_sumarizados.rename(columns={'total_TN': 'y'})

# Definir las columnas necesarias para NeuralProphet
expected_columns = ['ds', 'y']#, 'trimestre', 'avg_tn_por_producto', 'avg_qty_por_cliente']

# Filtrar datos por product_id
product_ids = datos_sumarizados['product_id'].unique()

# Preparar un DataFrame para almacenar los resultados
resultados = pd.DataFrame()

# Entrenar y predecir para cada product_id
for pid in product_ids:
    # Crear una nueva instancia del modelo para cada product_id
    model = NeuralProphet(
        n_lags=3,  # Número de lags para los regresores
        n_forecasts=2,  # Número de períodos a predecir
    )
    
    # Agregar los regresores laggeados
    #model.add_lagged_regressor("trimestre")
    #model.add_lagged_regressor("avg_tn_por_producto")
    #model.add_lagged_regressor("avg_qty_por_cliente")
    
    datos_producto = datos_sumarizados[datos_sumarizados['product_id'] == pid].copy()
    
    # Filtrar las columnas necesarias
    datos_producto = datos_producto[expected_columns]
    
    # Asegurarse de que 'ds' sea de tipo datetime
    datos_producto['ds'] = pd.to_datetime(datos_producto['ds'])
    
    # Ordenar los datos por fecha
    datos_producto = datos_producto.sort_values('ds')
    
    # Entrenar el modelo
    model.fit(datos_producto, freq='M')
    
    # Crear DataFrame futuro
    future = model.make_future_dataframe(df=datos_producto, periods=2)
    
    # Predecir
    forecast = model.predict(future)
    
    # Filtrar predicciones para febrero de 2020
    predicciones_febrero = forecast[forecast['ds'].dt.strftime('%Y-%m') == '2020-02'][['ds', 'yhat1']]
    
    # Agregar product_id a las predicciones
    predicciones_febrero['product_id'] = pid
    
    # Renombrar la columna de predicción
    predicciones_febrero.rename(columns={'yhat1': 'tn'}, inplace=True)
    
    # Añadir las predicciones al DataFrame de resultados
    resultados = pd.concat([resultados, predicciones_febrero])

# Guardar los resultados en un archivo CSV
resultados[['product_id', 'tn']].to_csv('predicciones_febrero_2020.v9.0.0.csv', index=False)

Finding best initial lr:   0%|          | 0/203 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

Predicting: 4it [00:00, ?it/s]

Finding best initial lr:   0%|          | 0/203 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

...

Training: 0it [00:00, ?it/s]

Predicting: 4it [00:00, ?it/s]

Finding best initial lr:   0%|          | 0/203 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

Predicting: 4it [00:00, ?it/s]